In [3]:
import pandas as pd

In [4]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('DATASET.csv')

# Display the first few rows to verify
df.head()

,event_id,session_id,user_id,variation,platform,datetime_event,event_type,final_order_status,shop_id
0,63527610,6391574,762832,2,Android,2024-11-30 16:23:46.391133804,entry_to_shop,successful,8531.0
1,12446736,6391574,762832,2,Android,2024-11-30 16:26:16.875713965,order_paid,successful,8531.0
2,90232698,6391574,762832,2,Android,2024-11-30 16:50:44.827088753,order_finished,successful,8531.0
3,74183469,9246026,762832,2,Android,2024-11-29 13:21:28.729203894,reload_the_page,NaN,6186.0
4,16360628,9246026,762832,2,Android,2024-11-29 13:21:35.213637901,entry_to_shop,NaN,6186.0


# Data Cleaning & EDA

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326921 entries, 0 to 326920
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   event_id            326921 non-null  int64  
 1   session_id          326921 non-null  int64  
 2   user_id             326921 non-null  int64  
 3   variation           326921 non-null  int64  
 4   platform            326921 non-null  object 
 5   datetime_event      326921 non-null  object 
 6   event_type          326921 non-null  object 
 7   final_order_status  170632 non-null  object 
 8   shop_id             315842 non-null  float64
dtypes: float64(1), int64(4), object(4)
memory usage: 22.4+ MB


In [9]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
event_id,326921.0,NaN,NaN,NaN,54971223.274348,26005727.141926,10000027.0,32451323.0,54919061.0,77504919.0,99999925.0
session_id,326921.0,NaN,NaN,NaN,5493266.772453,2598273.9833,1000058.0,3246170.0,5486265.0,7742072.0,9999979.0
user_id,326921.0,NaN,NaN,NaN,550159.16329,259722.459324,100003.0,325736.0,549509.0,775308.0,999985.0
variation,326921.0,NaN,NaN,NaN,1.349684,0.476871,1.0,1.0,1.0,2.0,2.0
platform,326921,2,Android,225929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
datetime_event,326921,324699,2024-11-29 23:59:24.000000000,674,NaN,NaN,NaN,NaN,NaN,NaN,NaN
event_type,326921,4,entry_to_shop,168215,NaN,NaN,NaN,NaN,NaN,NaN,NaN
final_order_status,170632,3,successful,146035,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shop_id,315842.0,NaN,NaN,NaN,5440.652238,2645.409437,1003.0,3150.0,5351.0,7801.0,9992.0


### **Summary Statistics**

**Mean of 1.35** - Variation 1 (control) has 65% of the data. (65%/35%) split between control and test isn't exactly even - was this intentional or a randomization error? 

Most events 69% occured on **Android devices** - why such a skew to one device? Will need to keep this in mind. 

In [ ]:
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_values


event_id                   0
session_id                 0
user_id                    0
variation                  0
platform                   0
datetime_event             0
event_type                 0
final_order_status    156289
shop_id                11079
dtype: int64

In [15]:
missing_percentage

event_id               0.000000
session_id             0.000000
user_id                0.000000
variation              0.000000
platform               0.000000
datetime_event         0.000000
event_type             0.000000
final_order_status    47.806351
shop_id                3.388892
dtype: float64

### **Missing Values**

Almost half of the events don't have a **final_order_status** - need to check if any order completion events are missing their status

**shop_id** is missing for some events - could be a data completeness issue. 